In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/corona-virus-report/usa_county_wise.csv
/kaggle/input/corona-virus-report/covid_19_clean_complete.csv


In [2]:
# thanks to:
#https://www.kaggle.com/shawon10/covid-19-easy-data-analysis-for-beginners
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
%matplotlib inline 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

In [3]:
#df=pd.read_csv('/kaggle/input/corona-virus-report/usa_county_wise.csv', na_filter=False)
df=pd.read_csv('/kaggle/input/corona-virus-report/usa_county_wise.csv', index_col='Date', parse_dates=True)
df=df.drop(['UID'],axis=1)
df.head(10)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-01-22,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",0,0
2020-01-22,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",0,0
2020-01-22,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",0,0
2020-01-22,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,"Puerto Rico, US",0,0
2020-01-22,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,"Virgin Islands, US",0,0
2020-01-22,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,0
2020-01-22,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",0,0
2020-01-22,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",0,0
2020-01-22,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",0,0


In [15]:
kansas_city_ks=df.loc[(df['Combined_Key'] == 'Wyandotte, Kansas, US') | 
                   (df['Combined_Key'] == 'Douglas, Kansas, US') | 
                   (df['Combined_Key'] == 'Johnson, Kansas, US')]
kansas_city_ks.tail(5)

,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-04-01,US,USA,840,20091.0,Johnson,Kansas,US,38.885482,-94.822509,"Johnson, Kansas, US",143,3
2020-04-01,US,USA,840,20209.0,Wyandotte,Kansas,US,39.117273,-94.763782,"Wyandotte, Kansas, US",93,4
2020-04-02,US,USA,840,20045.0,Douglas,Kansas,US,38.885036,-95.292512,"Douglas, Kansas, US",31,0
2020-04-02,US,USA,840,20091.0,Johnson,Kansas,US,38.885482,-94.822509,"Johnson, Kansas, US",161,3
2020-04-02,US,USA,840,20209.0,Wyandotte,Kansas,US,39.117273,-94.763782,"Wyandotte, Kansas, US",106,4


In [5]:
# Montgomery, maryland
montgomery_md=df.loc[df['Combined_Key'] == 'Montgomery, Maryland, US']
montgomery_md.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-29,US,USA,840,24031.0,Montgomery,Maryland,US,39.136763,-77.203582,"Montgomery, Maryland, US",301,1
2020-03-30,US,USA,840,24031.0,Montgomery,Maryland,US,39.136763,-77.203582,"Montgomery, Maryland, US",341,1
2020-03-31,US,USA,840,24031.0,Montgomery,Maryland,US,39.136763,-77.203582,"Montgomery, Maryland, US",388,1
2020-04-01,US,USA,840,24031.0,Montgomery,Maryland,US,39.136763,-77.203582,"Montgomery, Maryland, US",447,1
2020-04-02,US,USA,840,24031.0,Montgomery,Maryland,US,39.136763,-77.203582,"Montgomery, Maryland, US",498,6


In [7]:
#columbus, ohio - Franklin Delaware Fairfield 
columbus_oh=df.loc[(df['Combined_Key'] == 'Franklin, Ohio, US') | 
                   (df['Combined_Key'] == 'Delaware, Ohio, US') | 
                   (df['Combined_Key'] == 'Fairfield, Ohio, US')]
columbus_oh.tail(5)

,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-04-01,US,USA,840,39045.0,Fairfield,Ohio,US,39.751072,-82.630882,"Fairfield, Ohio, US",22,0
2020-04-01,US,USA,840,39049.0,Franklin,Ohio,US,39.969958,-83.011158,"Franklin, Ohio, US",391,4
2020-04-02,US,USA,840,39041.0,Delaware,Ohio,US,40.279424,-83.004571,"Delaware, Ohio, US",50,1
2020-04-02,US,USA,840,39045.0,Fairfield,Ohio,US,39.751072,-82.630882,"Fairfield, Ohio, US",28,0
2020-04-02,US,USA,840,39049.0,Franklin,Ohio,US,39.969958,-83.011158,"Franklin, Ohio, US",441,5


In [4]:
#youngstown, ohio - Mahoning
youngstown_oh=df.loc[df['Admin2'] == 'Mahoning']
youngstown_oh.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-29,US,USA,840,39099.0,Mahoning,Ohio,US,41.016311,-80.77287,"Mahoning, Ohio, US",84,2
2020-03-30,US,USA,840,39099.0,Mahoning,Ohio,US,41.016311,-80.77287,"Mahoning, Ohio, US",117,4
2020-03-31,US,USA,840,39099.0,Mahoning,Ohio,US,41.016311,-80.77287,"Mahoning, Ohio, US",153,9
2020-04-01,US,USA,840,39099.0,Mahoning,Ohio,US,41.016311,-80.77287,"Mahoning, Ohio, US",177,10
2020-04-02,US,USA,840,39099.0,Mahoning,Ohio,US,41.016311,-80.77287,"Mahoning, Ohio, US",192,11


In [6]:
# Pittsburg - Allegheny
pittsburg_pa=df.loc[df['Admin2'] == 'Allegheny']
pittsburg_pa.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-29,US,USA,840,42003.0,Allegheny,Pennsylvania,US,40.468099,-79.981677,"Allegheny, Pennsylvania, US",265,2
2020-03-30,US,USA,840,42003.0,Allegheny,Pennsylvania,US,40.468099,-79.981677,"Allegheny, Pennsylvania, US",290,2
2020-03-31,US,USA,840,42003.0,Allegheny,Pennsylvania,US,40.468099,-79.981677,"Allegheny, Pennsylvania, US",325,2
2020-04-01,US,USA,840,42003.0,Allegheny,Pennsylvania,US,40.468099,-79.981677,"Allegheny, Pennsylvania, US",356,2
2020-04-02,US,USA,840,42003.0,Allegheny,Pennsylvania,US,40.468099,-79.981677,"Allegheny, Pennsylvania, US",419,2


In [8]:
# Haymarket - Prince William
haymarket_va=df.loc[df['Admin2'] == 'Prince William']
haymarket_va.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-29,US,USA,840,51153.0,Prince William,Virginia,US,38.709162,-77.487137,"Prince William, Virginia, US",72,1
2020-03-30,US,USA,840,51153.0,Prince William,Virginia,US,38.709162,-77.487137,"Prince William, Virginia, US",79,1
2020-03-31,US,USA,840,51153.0,Prince William,Virginia,US,38.709162,-77.487137,"Prince William, Virginia, US",94,1
2020-04-01,US,USA,840,51153.0,Prince William,Virginia,US,38.709162,-77.487137,"Prince William, Virginia, US",106,1
2020-04-02,US,USA,840,51153.0,Prince William,Virginia,US,38.709162,-77.487137,"Prince William, Virginia, US",117,1


In [9]:
#Virginia Beach, Virginia
virginia_beach_va=df.loc[(df['Combined_Key'] == 'Virginia Beach, Virginia, US') | 
                         (df['Combined_Key'] == 'Chesapeake, Virginia, US') | 
                         (df['Combined_Key'] == 'Norfolk, Virginia, US')]
virginia_beach_va.tail(5)
#Chesapeake, Virginia
#chesapeake_va=df.loc[df['Combined_Key'] == 'Chesapeake, Virginia, US']
#chesapeake_va.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-04-01,US,USA,840,51710.0,Norfolk,Virginia,US,36.899340,-76.264209,"Norfolk, Virginia, US",33,0
2020-04-01,US,USA,840,51810.0,Virginia Beach,Virginia,US,36.732857,-76.045669,"Virginia Beach, Virginia, US",88,1
2020-04-02,US,USA,840,51550.0,Chesapeake,Virginia,US,36.676700,-76.305457,"Chesapeake, Virginia, US",39,0
2020-04-02,US,USA,840,51710.0,Norfolk,Virginia,US,36.899340,-76.264209,"Norfolk, Virginia, US",37,0
2020-04-02,US,USA,840,51810.0,Virginia Beach,Virginia,US,36.732857,-76.045669,"Virginia Beach, Virginia, US",111,1


In [10]:
#king, WA
seatle_wa=df.loc[(df['Combined_Key'] == 'King, Washington, US') | 
                    (df['Combined_Key'] == 'Snohomish, Washington, US')]
#seatle_wa.head(5) # 1/22/20
seatle_wa.tail(5) # 4/1/20


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-31,US,USA,840,53061.0,Snohomish,Washington,US,48.046160,-121.717070,"Snohomish, Washington, US",1229,33
2020-04-01,US,USA,840,53033.0,King,Washington,US,47.491379,-121.834613,"King, Washington, US",2330,150
2020-04-01,US,USA,840,53061.0,Snohomish,Washington,US,48.046160,-121.717070,"Snohomish, Washington, US",1304,40
2020-04-02,US,USA,840,53033.0,King,Washington,US,47.491379,-121.834613,"King, Washington, US",2656,175
2020-04-02,US,USA,840,53061.0,Snohomish,Washington,US,48.046160,-121.717070,"Snohomish, Washington, US",1376,41


In [11]:
#Snohomish, WA
snohomish_wa=df.loc[(df['Combined_Key'] == 'Snohomish, Washington, US')]
snohomish_wa.tail(5)


,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Deaths
Date,,,,,,,,,,,,
2020-03-29,US,USA,840,53061.0,Snohomish,Washington,US,48.04616,-121.71707,"Snohomish, Washington, US",912,23
2020-03-30,US,USA,840,53061.0,Snohomish,Washington,US,48.04616,-121.71707,"Snohomish, Washington, US",1068,23
2020-03-31,US,USA,840,53061.0,Snohomish,Washington,US,48.04616,-121.71707,"Snohomish, Washington, US",1229,33
2020-04-01,US,USA,840,53061.0,Snohomish,Washington,US,48.04616,-121.71707,"Snohomish, Washington, US",1304,40
2020-04-02,US,USA,840,53061.0,Snohomish,Washington,US,48.04616,-121.71707,"Snohomish, Washington, US",1376,41


In [12]:
#import dateutil
#data['Date'] = df['Date'].apply(dateutil.parser.parse, dayfirst=True)
#df['date'] = pd.to_datetime(df['Date'])
group = df.groupby('Date')['Confirmed'].sum().reset_index()
#group = df.groupby('Date')['Confirmed'].sum().reset_index()
#group = df.groupby(['Date'].date())['Confirmed'].sum().reset_index()
#group = df.groupby(df,by=[df['Date'].date()['Confirmed'].sum().reset_index()

fig = px.line(group, x="Date", y="Confirmed", 
              title="US Confirmed Cases Over Time")

fig.show()

In [13]:
group = youngstown_oh.groupby('Date')['Confirmed'].sum().reset_index()

fig = px.line(group, x="Date", y="Confirmed", 
              title="Youngstown, OH Confirmed Cases Over Time")

fig.show()

In [16]:
group = kansas_city_ks.groupby('Date')['Confirmed'].sum().reset_index()

fig = px.line(group, x="Date", y="Confirmed", 
              title="Kansas City Confirmed Cases Over Time")

fig.show()

In [18]:
#kansas_city_ks.sort_values(by=['Date'],ascending=True).plot(x='Date',y='Confirmed',fontsize=15,figsize=(12,18), kind='barh');

KeyError: 'Date'